In [1]:
import requests
from requests.exceptions import ConnectionError
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
companies1 = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1") #unmodified names
companies = pd.read_csv('List_of_companies_Full.csv', encoding = "ISO-8859-1") #will have modified names

print(companies.dtypes)

companies['Security'] = companies['Security'].str.replace(u'\xa0', u' ')#this was messing with the classes
companies['Security'] = companies['Security'].str.replace(r' \(Class A\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class B\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class C\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r'\(', '')#need to remove parenthesis too for the url
companies['Security'] = companies['Security'].str.replace(r'\)', '')
companies['Security'] = companies['Security'].str.replace(r'\.com', '-com')#for amazon and salesforce
companies['Security'] = companies['Security'].str.replace(r'Corporation', 'corp')#corporation got shortened to corp
companies['Security'] = companies['Security'].str.replace(r'corporation', 'corp')
companies['Security'] = companies['Security'].str.replace(r'Company', 'co')#company got shortened to co
companies['Security'] = companies['Security'].str.replace(r'company', 'co')
companies['Security'] = companies['Security'].str.replace(r'&', 'and')#& is and for some reason
companies['Security'] = companies['Security'].str.replace(r' ', '-')#the url uses hyphens a lot
companies['Security'] = companies['Security'].str.replace(r'.', '')#don't want these here
companies['Security'] = companies['Security'].str.replace(r',', '')
companies['Security'] = companies['Security'].str.replace(r'!', '')
companies['Security'] = companies['Security'].str.replace(r'\'', '')

companies['Security'] = companies['Security'].str.lower()#lowercases everything
print(len(companies))
companies

Symbol                   object
Security                 object
SEC filings              object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date first added         object
CIK                       int64
Founded                  object
dtype: object
500


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3m-co,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,abbott-laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,abbvie-inc,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,abiomed-inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,accenture-plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
5,ATVI,activision-blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",8/31/2015,718877,2008
6,ADBE,adobe-inc,reports,Information Technology,Application Software,"San Jose, California",5/5/1997,796343,1982
7,AMD,advanced-micro-devices-inc,reports,Information Technology,Semiconductors,"Santa Clara, California",3/20/2017,2488,1969
8,AAP,advance-auto-parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",7/9/2015,1158449,1932
9,AES,aes-corp-the,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",10/2/1998,874761,1981


In [3]:
#getting url this is for testing purposes
for n in range(10):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    print(url)


https://www1.salary.com/3m-co-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
https://www1.salary.com/abbvie-inc-Executive-Salaries.html
https://www1.salary.com/abiomed-inc-Executive-Salaries.html
https://www1.salary.com/accenture-plc-Executive-Salaries.html
https://www1.salary.com/activision-blizzard-Executive-Salaries.html
https://www1.salary.com/adobe-inc-Executive-Salaries.html
https://www1.salary.com/advanced-micro-devices-inc-Executive-Salaries.html
https://www1.salary.com/advance-auto-parts-Executive-Salaries.html
https://www1.salary.com/aes-corp-the-Executive-Salaries.html


In [4]:
#utilities
def getUrl(n, mod = ''):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + mod + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getIncUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-inc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCoUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-co' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getCorpUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-corp' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getPlcUrl(n):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-plc' + '-Executive-Salaries.html'
    page=requests.get(url)
    return page

def getContent(page):
    doc=lh.fromstring(page.content)
    tr_elements=doc.xpath('//tr')
    return tr_elements

In [5]:
def failureCase(col, good, alt_tr_elements):
    for j in range(0,len(alt_tr_elements)):
        #T is our j'th row
        T=alt_tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                T=alt_tr_elements[j+1]
                #i is the index of our column
                i=0

                #this is a temporary storage of the content of each line of the csv
                content = []

                #Iterate through each element of the row
                for t in T.iterchildren():
                    #this part if for cleaning the cell contents
                    data=t.text_content() #gets the content of cell
                    data = data.__str__() #turns content to string
                    data = data.replace(r'$', '') #takes out dollar sign for number conversion
                    data = data.replace(r',', '') #takes out comma for number conversion
                    data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                    data = data.replace(r'CEO Pay', '') #cleans up the content string
                    data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                    data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                    data = data.replace(r'Ratio', '') #cleans up the content string
                    data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                    data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                    content.append(data)

                    #Check if row is empty
                    if i>0:
                    #Convert any numerical value to integers
                        try:
                            data=int(data)
                        except:
                            pass
                    #Append the data to col
                    col[i + 1][1].append(data)
                    #Increment i for the next column
                    i+=1
                #col[0][1].append(currName1)
                return True

        return False

In [6]:
#Makes a list of to make the graph later on
col=[('Company Name', []), ('CEO Name', []), ('CEO Pay', []), ('Median Employee Pay', []), ('CEO Pay Ratio', [])]

In [7]:
#goes through all the webpages
for n in range(len(companies)):
    currName = companies.at[n, 'Security']
    currName1 =companies1.at[n, 'Security']
    col[0][1].append(currName1)
    #getting url
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    page=requests.get(url)
    #getting all the tables
    tr_elements=getContent(page)
    #indicates that we failed to find the magics on this particular page
    fail = True
    #skip = False
    """try:
            doc=lh.fromstring(page.content)
            tr_elements=doc.xpath('//tr')
        except:
            skip = True"""
    """if skip:
            print(1)
            print(currName1)
            print(currName)
            print(url)
            col[1][1].append('')
            col[2][1].append(-1)
            col[3][1].append(-1)
            col[4][1].append(-1)"""
    #this will find the table that has the relevant info for the CEO,and put the correct info into col for this one page
    for j in range(0,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
        #and CEO Pay Ratio)
        if len(T)==4:
            #checks if the current row has the 4 maagic words
            table_Name = ['','','','']
            i=0
            for t in T:
                name = t.text_content()
                name = name.__str__()
                table_Name[i]=name
                i+=1

            #if it does, I'll take the data from the row right below it
            if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                #print(currName1 + ' good')
                #setting fail to false
                fail = False
                T=tr_elements[j+1]
                #i is the index of our column
                i=0

                #this is a temporary storage of the content of each line of the csv
                content = []

                #Iterate through each element of the row
                for t in T.iterchildren():
                    #this part if for cleaning the cell contents
                    data=t.text_content() #gets the content of cell
                    data = data.__str__() #turns content to string
                    data = data.replace(r'$', '') #takes out dollar sign for number conversion
                    data = data.replace(r',', '') #takes out comma for number conversion
                    data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                    data = data.replace(r'CEO Pay', '') #cleans up the content string
                    data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                    data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                    data = data.replace(r'Ratio', '') #cleans up the content string
                    data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                    data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                    content.append(data)

                    #Check if row is empty
                    if i>0:
                    #Convert any numerical value to integers
                        try:
                            data=int(data)
                        except:
                            pass
                    #Append the data to col
                    col[i + 1][1].append(data)
                    #Increment i for the next column
                    i+=1
                #col[0][1].append(currName1)
                break
    
    if fail:
        #print(currName1)
        #print(currName)
        #print(url)
        #put these into the get url
        alts = ['-inc','-co','-corp','-plc','-new']
        #good indicated whether or not the new url works
        good = False
        for i in range(len(alts)):
            url='https://www1.salary.com/' + currName + alts[i] + '-Executive-Salaries.html'
            page=requests.get(url)
            alt_tr_elements=getContent(page)
            for j in range(0,len(alt_tr_elements)):
                #T is our j'th row
                T=alt_tr_elements[j]

                #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
                #and CEO Pay Ratio)
                if len(T)==4:
                    #checks if the current row has the 4 maagic words
                    table_Name = ['','','','']
                    i=0
                    for t in T:
                        name = t.text_content()
                        name = name.__str__()
                        table_Name[i]=name
                        i+=1

                    #if it does, I'll take the data from the row right below it
                    if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                        #print(currName1 + ' good')
                        T=alt_tr_elements[j+1]
                        good = True
                        #i is the index of our column
                        i=0

                        #this is a temporary storage of the content of each line of the csv
                        content = []

                        #Iterate through each element of the row
                        for t in T.iterchildren():
                            #this part if for cleaning the cell contents
                            data=t.text_content() #gets the content of cell
                            data = data.__str__() #turns content to string
                            data = data.replace(r'$', '') #takes out dollar sign for number conversion
                            data = data.replace(r',', '') #takes out comma for number conversion
                            data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                            data = data.replace(r'CEO Pay', '') #cleans up the content string
                            data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                            data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                            data = data.replace(r'Ratio', '') #cleans up the content string
                            data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                            data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                            content.append(data)

                            #Check if row is empty
                            if i>0:
                            #Convert any numerical value to integers
                                try:
                                    data=int(data)
                                except:
                                    pass
                            #Append the data to col
                            col[i + 1][1].append(data)
                            #Increment i for the next column
                            i+=1
                        #col[0][1].append(currName1)
            if good:
                break
            #else:
                #print(url)
        #this means the page failed
        if (good == False):
            col[1][1].append('na')
            col[2][1].append(-1)
            col[3][1].append(-1)
            col[4][1].append(-1)
            print(n, url)
    print(n)
    

0
1 https://www1.salary.com/abbott-laboratories-new-Executive-Salaries.html
1
2
3
4 https://www1.salary.com/accenture-plc-new-Executive-Salaries.html
4
5
6
7
8
9
10
11
12
13
14
15
16 https://www1.salary.com/alexandria-real-estate-equities-new-Executive-Salaries.html
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70 https://www1.salary.com/boeing-co-new-Executive-Salaries.html
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87 https://www1.salary.com/carrier-global-new-Executive-Salaries.html
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160 https://www1.salary.com/ebay-inc-new-Executive-Salaries.html
160
161
162
163
164
165
166
167
168
169
1

In [8]:
col

[('Company Name',
  ['3M Company',
   'Abbott Laboratories',
   'AbbVie Inc.',
   'ABIOMED Inc',
   'Accenture plc',
   'Activision Blizzard',
   'Adobe Inc.',
   'Advanced Micro Devices Inc',
   'Advance Auto Parts',
   'AES Corp',
   'AFLAC Inc',
   'Agilent Technologies Inc',
   'Air Products & Chemicals Inc',
   'Akamai Technologies Inc',
   'Alaska Air Group Inc',
   'Albemarle Corp',
   'Alexandria Real Estate Equities',
   'Alexion Pharmaceuticals',
   'Align Technology',
   'Allegion',
   'Alliant Energy Corp',
   'Allstate Corp',
   'Alphabet Inc.',
   'Altria Group Inc',
   'Amazon.com Inc.',
   'Amcor plc',
   'Ameren Corp',
   'American Airlines Group',
   'American Electric Power',
   'American Express Co',
   'American International Group',
   'American Tower Corp.',
   'American Water Works Company Inc',
   'Ameriprise Financial',
   'AmerisourceBergen Corp',
   'AMETEK Inc.',
   'Amgen Inc.',
   'Amphenol Corp',
   'Analog Devices, Inc.',
   'ANSYS',
   'Anthem',
   'Ao

In [9]:
print(len(col[0][1]))
#print(col[0][1].pop())
#col[0][1].append('VF Corporation')
print(len(col[0][1]))
print(len(col[1][1]))
print(len(col[2][1]))
print(len(col[3][1]))

500
500
500
500
500


In [10]:
#making the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [11]:
#displaying the dataframe
df

,Company Name,CEO Name,CEO Pay,Median Employee Pay,CEO Pay Ratio
0,3M Company,Michael F. Roman,18321566,57494,319
1,Abbott Laboratories,na,-1,-1,-1
2,AbbVie Inc.,Richard A. Gonzalez,21610598,155885,139
3,ABIOMED Inc,Michael R. Minogue,16612766,112393,148
4,Accenture plc,na,-1,-1,-1
5,Activision Blizzard,Robert A. Kotick,30122896,94308,319
6,Adobe Inc.,Shantanu Narayen,39145631,147115,266
7,Advanced Micro Devices Inc,Lisa T. Su,58534288,96874,604
8,Advance Auto Parts,Thomas R. Greco,7691513,19463,395
9,AES Corp,Andrés Gluski,10827886,61975,175


In [12]:
int = 0
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] > -1):
        int += 1
print(int)
print(len(df)-int)

480
20


int = 0
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] == -1):
        int += 1
        print(i)
        print(df.at[i, 'Company Name'])
int

In [13]:
int = 0
for i in range(len(df)):
    if ('ney' in df.at[i, 'Company Name']):
        int += 1
        print(i)
int

230
476


2

In [14]:
df.at[230, 'Company Name']

"Honeywell Int'l Inc."

In [15]:
companies.at[488, 'Security']

'williams-cos-inc'

In [16]:
companies1.at[487, 'Security']

'Whirlpool Corp.'

In [17]:
#verifying correctness

int = 0

for i in range(len(companies)):
    if (df.at[i, 'Company Name'] != companies1.at[i, 'Security']):
        int += 1
        print(df.at[i, 'Company Name'] + " is not " + companies1.at[i, 'Security'])
        print(i)
int

0

In [18]:
#adding industry
print(list(companies1.columns))
industries = list(companies1['GICS\xa0Sector'])
('GICS Sector', industries)

['Symbol', 'Security', 'SEC filings', 'GICS\xa0Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK', 'Founded']


('GICS Sector',
 ['Industrials',
  'Health Care',
  'Health Care',
  'Health Care',
  'Information Technology',
  'Communication Services',
  'Information Technology',
  'Information Technology',
  'Consumer Discretionary',
  'Utilities',
  'Financials',
  'Health Care',
  'Materials',
  'Information Technology',
  'Industrials',
  'Materials',
  'Real Estate',
  'Health Care',
  'Health Care',
  'Industrials',
  'Utilities',
  'Financials',
  'Communication Services',
  'Consumer Staples',
  'Consumer Discretionary',
  'Materials',
  'Utilities',
  'Industrials',
  'Utilities',
  'Financials',
  'Financials',
  'Real Estate',
  'Utilities',
  'Financials',
  'Health Care',
  'Industrials',
  'Health Care',
  'Information Technology',
  'Information Technology',
  'Information Technology',
  'Health Care',
  'Financials',
  'Industrials',
  'Energy',
  'Real Estate',
  'Information Technology',
  'Information Technology',
  'Consumer Discretionary',
  'Consumer Staples',
  'Information

In [19]:
subIndustries = list(companies1['GICS Sub-Industry'])
('GICS Sub-Industry', subIndustries)

('GICS Sub-Industry',
 ['Industrial Conglomerates',
  'Health Care Equipment',
  'Pharmaceuticals',
  'Health Care Equipment',
  'IT Consulting & Other Services',
  'Interactive Home Entertainment',
  'Application Software',
  'Semiconductors',
  'Automotive Retail',
  'Independent Power Producers & Energy Traders',
  'Life & Health Insurance',
  'Health Care Equipment',
  'Industrial Gases',
  'Internet Services & Infrastructure',
  'Airlines',
  'Specialty Chemicals',
  'Office REITs',
  'Biotechnology',
  'Health Care Supplies',
  'Building Products',
  'Electric Utilities',
  'Property & Casualty Insurance',
  'Interactive Media & Services',
  'Tobacco',
  'Internet & Direct Marketing Retail',
  'Paper Packaging',
  'Multi-Utilities',
  'Airlines',
  'Electric Utilities',
  'Consumer Finance',
  'Property & Casualty Insurance',
  'Specialized REITs',
  'Water Utilities',
  'Asset Management & Custody Banks',
  'Health Care Distributors',
  'Electrical Components & Equipment',
  'Bi

In [20]:
col.append(('GICS Sector', industries))
col

[('Company Name',
  ['3M Company',
   'Abbott Laboratories',
   'AbbVie Inc.',
   'ABIOMED Inc',
   'Accenture plc',
   'Activision Blizzard',
   'Adobe Inc.',
   'Advanced Micro Devices Inc',
   'Advance Auto Parts',
   'AES Corp',
   'AFLAC Inc',
   'Agilent Technologies Inc',
   'Air Products & Chemicals Inc',
   'Akamai Technologies Inc',
   'Alaska Air Group Inc',
   'Albemarle Corp',
   'Alexandria Real Estate Equities',
   'Alexion Pharmaceuticals',
   'Align Technology',
   'Allegion',
   'Alliant Energy Corp',
   'Allstate Corp',
   'Alphabet Inc.',
   'Altria Group Inc',
   'Amazon.com Inc.',
   'Amcor plc',
   'Ameren Corp',
   'American Airlines Group',
   'American Electric Power',
   'American Express Co',
   'American International Group',
   'American Tower Corp.',
   'American Water Works Company Inc',
   'Ameriprise Financial',
   'AmerisourceBergen Corp',
   'AMETEK Inc.',
   'Amgen Inc.',
   'Amphenol Corp',
   'Analog Devices, Inc.',
   'ANSYS',
   'Anthem',
   'Ao

In [21]:
col.append(('GICS Sub-Industry', subIndustries))
col

[('Company Name',
  ['3M Company',
   'Abbott Laboratories',
   'AbbVie Inc.',
   'ABIOMED Inc',
   'Accenture plc',
   'Activision Blizzard',
   'Adobe Inc.',
   'Advanced Micro Devices Inc',
   'Advance Auto Parts',
   'AES Corp',
   'AFLAC Inc',
   'Agilent Technologies Inc',
   'Air Products & Chemicals Inc',
   'Akamai Technologies Inc',
   'Alaska Air Group Inc',
   'Albemarle Corp',
   'Alexandria Real Estate Equities',
   'Alexion Pharmaceuticals',
   'Align Technology',
   'Allegion',
   'Alliant Energy Corp',
   'Allstate Corp',
   'Alphabet Inc.',
   'Altria Group Inc',
   'Amazon.com Inc.',
   'Amcor plc',
   'Ameren Corp',
   'American Airlines Group',
   'American Electric Power',
   'American Express Co',
   'American International Group',
   'American Tower Corp.',
   'American Water Works Company Inc',
   'Ameriprise Financial',
   'AmerisourceBergen Corp',
   'AMETEK Inc.',
   'Amgen Inc.',
   'Amphenol Corp',
   'Analog Devices, Inc.',
   'ANSYS',
   'Anthem',
   'Ao

In [22]:
Dict={title:column for (title,column) in col}
df_ratio=pd.DataFrame(Dict)

In [23]:
df_ratio

,Company Name,CEO Name,CEO Pay,Median Employee Pay,CEO Pay Ratio,GICS Sector,GICS Sub-Industry
0,3M Company,Michael F. Roman,18321566,57494,319,Industrials,Industrial Conglomerates
1,Abbott Laboratories,na,-1,-1,-1,Health Care,Health Care Equipment
2,AbbVie Inc.,Richard A. Gonzalez,21610598,155885,139,Health Care,Pharmaceuticals
3,ABIOMED Inc,Michael R. Minogue,16612766,112393,148,Health Care,Health Care Equipment
4,Accenture plc,na,-1,-1,-1,Information Technology,IT Consulting & Other Services
5,Activision Blizzard,Robert A. Kotick,30122896,94308,319,Communication Services,Interactive Home Entertainment
6,Adobe Inc.,Shantanu Narayen,39145631,147115,266,Information Technology,Application Software
7,Advanced Micro Devices Inc,Lisa T. Su,58534288,96874,604,Information Technology,Semiconductors
8,Advance Auto Parts,Thomas R. Greco,7691513,19463,395,Consumer Discretionary,Automotive Retail
9,AES Corp,Andrés Gluski,10827886,61975,175,Utilities,Independent Power Producers & Energy Traders


In [24]:
#verifying correctness

int = 0

for i in range(len(companies)):
    if (df_ratio.at[i, 'GICS Sector'] != companies1.at[i, 'GICS\xa0Sector'] or df_ratio.at[i, 'GICS Sub-Industry'] != companies1.at[i, 'GICS Sub-Industry']):
        int += 1
        print(df.at[i, 'Company Name'] + " is not " + companies1.at[i, 'Security'])
        print(i)
int

0

In [25]:
#df.to_csv('official_ratios.csv',index=False)

In [26]:
#df_ratio.to_csv('official_ratio_with_industries.csv',index=False)

In [27]:
int = []
for i in range(len(df)):
    if (df.at[i, 'Median Employee Pay'] == -1):
        int.append(i)
print(len(int))
int

20


[1,
 4,
 16,
 70,
 87,
 160,
 178,
 190,
 203,
 212,
 255,
 292,
 332,
 352,
 353,
 368,
 396,
 402,
 423,
 432]